
<h1 style="text-align: center;"><b>Лабараторная работа № 2. K-Nearest Neighbors algorithm</b></h1>

   
Классификатор K ближайших соседей (KNN) является одним из самых простых классификаторов в обучении с учителем. KNN считается ленивым учеников;он технически не тренирует модель делать предсказания. Вместо этого предсказывается, что наблюдение будет классом наибольшей доли k ближайших наблюдений. 
<p style="align: center;"><img src="https://www.researchgate.net/profile/Saleh_Alaliyat/publication/267953942/figure/fig14/AS:295388776026147@1447437580523/K-nearest-neighbor-algorithm-illustration-The-green-circle-is-the-sample-which-is-to-be.png" width="400"></p>
   
Например, если наблюдение с неизвестным классом окружено наблюдением класса 1, то это наблюдение классифицируется как класс 1.


In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
lib = {'scikit-learn' : sklearn, 'pandas' : pd, 'numpy' : np}
for i in lib:
    print(f'{i} version :{lib[i].__version__}')

/home/khoreks/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


scikit-learn version :0.23.2
pandas version :1.1.2
numpy version :1.19.2


### Загрузим данные

[Скачать данные](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).   
Или можно загрузить их из библиотеки sklearn

In [2]:
# Загрузка данных
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y = True, as_frame=True)
print(load_breast_cancer()['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [3]:
X.shape #размерность данных

(569, 30)

С помощью метода value_counts посмотрим количество уникальных даннных содержащихся target данных.

In [4]:
y.value_counts()

1    357
0    212
Name: target, dtype: int64

Исходя из данных представленных выше, можно сделать вывод, что мы имеем дело с бинарным классификатором.

Посмотрим на данные

In [5]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
X.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


Для обучения KNN классификатора нужно выбрать количество соседей на которые мы будет опираться при выборе класса.   
В нашем случае мы используем следующие параметры: 1, 5, 10, 25, 50, 100.   

В данном алгоритме расстояние рассматривается как мера сходства. Библиотека scikit-learn предлагает на выбор метрические показатели расстояния d.   

Нами будут использоваться:
* Евклидово расстояние : $$ \mathbf{d_{евклидово}} = {\sqrt{\sum_{i=1}^n {(x_i-y_i)}^2}}$$
* Манхэттенское расстояние : $$ \mathbf{d_{манхэттенское}} = {\sum_{i=1}^n {|x_i-y_i|}}$$
* Чебышева расстояние : $$\mathbf{d_{чебышева}(x,y)} = max_i (|x_i - y_i|)$$ 
* Минковского расстояние : $$ \mathbf{d_{минковского}} = ({\sum_{i=1}^n {|x_i-y_i|}^p})^{1/p}$$

In [9]:
K = [1, 5, 10, 25, 50, 100]
metrics = ['euclidean','manhattan', 'chebyshev', 'minkowski']

* Разбивать массивы или матрицы на случайные обучающие и тестовые подмножества будем при помощи метода train_test_split библиотеки scikit-learn.
* Создадим и натренируем классификатор используя число наблюдений и метрики представленные выше используя 
KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)
* Оценим результат перекрестной проверкой используя cross_val_score.
* Результаты запишем в DataFrame

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

df = pd.DataFrame(index=metrics)
for k_n in K:
    score = []
    for k_metric in metrics:
        clf = KNeighborsClassifier(n_neighbors=k_n, metric=k_metric)
        score.append(cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5).mean())
    df['n_neighbors:' + str(k_n)] = score
df

,n_neighbors:1,n_neighbors:5,n_neighbors:10,n_neighbors:25,n_neighbors:50,n_neighbors:100
euclidean,0.920082,0.927086,0.938851,0.915376,0.915404,0.894337
manhattan,0.927141,0.941204,0.943557,0.920082,0.908372,0.889631
chebyshev,0.922462,0.927114,0.934145,0.913023,0.915404,0.896662
minkowski,0.920082,0.927086,0.938851,0.915376,0.915404,0.894337


Лучашя метрика manhattan с n_neighbors:10   
Расчитаем accuracy для тестовой выборки у KNeighborsClassifier c найденными параметрами.

In [16]:
clf = KNeighborsClassifier(n_neighbors=10, metric='manhattan')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9440559440559441

## GridSearchCV

GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

Традиционным способом выполнения гиперпараметрической оптимизации является поиск по сетке или поиск по параметрам , который представляет собой просто исчерпывающий поиск по заданному вручную подмножеству гиперпараметрического пространства алгоритма обучения. Алгоритм поиска по сетке должен руководствоваться некоторой метрикой производительности, обычно измеряемой перекрестной проверкой на обучающем наборе

In [27]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': range(1,11),
    'metric': ['euclidean','manhattan', 'chebyshev', 'minkowski']
}

gs_knn = GridSearchCV(KNeighborsClassifier(),
                     grid_params,
                     cv=5)
gs_knn.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'chebyshev',
                                    'minkowski'],
                         'n_neighbors': range(1, 11)})

In [28]:
print(f'Best score : {gs_knn.best_score_}')
print(f'Best parametrs : {gs_knn.best_params_}')

Best score : 0.9459097127222982
Best parametrs : {'metric': 'manhattan', 'n_neighbors': 9}


Таким образом оптимальными гиперпараметрами для "breast cancer"выборки являются: metric = manhattan, n_neighbors = 9.

In [25]:
new_clf = KNeighborsClassifier(n_neighbors=9, metric='manhattan')
new_clf.fit(X_train, y_train)
new_clf.score(X_test, y_test)

0.958041958041958

После применения GridSearchCV были найдены оптимальные параметры, что привело к увеличению точность модели на тестовой выборке до 0.958041958041958